# Tying up some loose ends

I can't let you out of this course without telling you about merging and the power of group_by/mutate to flexibly group by different factors in the same data-frame. 

In [2]:
library(tidyverse)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.7     v dplyr   1.0.8
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Let's load in some data. This is a sample of a large set of iconicity word norms. Participant see a list of words and rate each one on a 1-7 scale. A high rating indicates the sound of the word has a resemblance to its meanings. A low rating indicates it does not. We have multiple ratings for each word. We have two files -- one containing ratings and the other containing some demographic infrmation -- a participant's gender and age. 

In [24]:
responses <- read.csv("https://psych750.github.io/data/iconicity_sample.csv")
demographics <- read.csv("https://psych750.github.io/data/iconicity_sample_demographics.csv")


In [25]:
responses %>% arrange(subj_code) %>% head

,subj_code,word,key,rt
,<int>,<fct>,<int>,<int>
1,1,abdominal,1,3245
2,1,select,1,3335
3,1,grain,4,2612
4,1,clothes,5,2500
5,1,hepatitis,1,2174
6,1,carousel,3,2336


In [26]:
demographics %>% head


## Merging



In [40]:
left_join(convert_gender,demographics) %>% head


Joining, by = "gender"


,gender,gender2,participant_code,age
,<fct>,<fct>,<int>,<dbl>
1,male,man,4,47.58836
2,male,man,6,40.85812
3,male,man,7,43.56354
4,male,man,9,32.45972
5,male,man,12,41.60187
6,male,man,15,40.81981


In [34]:
library(psychTools)
convert_gender =read.clipboard.csv(sep="\t")

Warning message in read.table(pipe("pbpaste"), header = TRUE, sep, ...):
"incomplete final line found by readTableHeader on 'pbpaste'"


In [35]:
convert_gender

gender,gender2
<fct>,<fct>
male,man
female,woman


Suppose we want to merge these two files. What do we do?

## Getting group-level variables

We've frequently used `group_by` in combination with `summarize`. But `dplyr` allows us to also use it in combination with `mutate` to do some pretty magical stuff. For example, suppose we want to add in to our responses data-frame the median reaction time for each subject. Easy! 

In [41]:
responses %>% group_by(word) %>% summarize(mean_resp=mean(key,na.rm=TRUE)) %>% head

word,mean_resp
<fct>,<dbl>
American,7.000000
Americanize,3.000000
April,2.500000
Aurora,3.333333
Baggie,3.000000
Banning,7.000000


In [9]:
responses %>% group_by(subj_code) %>% mutate(median_RT = median(rt)) %>% head(25)

subj_code,word,key,rt,median_RT
<int>,<fct>,<int>,<int>,<dbl>
1,abdominal,1,3245,2449
1,select,1,3335,2449
1,grain,4,2612,2449
1,clothes,5,2500,2449
1,hepatitis,1,2174,2449
1,carousel,3,2336,2449
1,seer,1,2093,2449
1,reek,1,3116,2449
1,pamper,1,1856,2449


This pattern is especially useful to create by-factor normalized variables, e.g.

In [20]:
responses %>% 
   group_by(subj_code) %>% 
   mutate(scaled_resp_by_subj = scale(key),num_ratings_by_subj=n()) %>% 
   group_by(word) %>% 
   mutate(scaled_resp_by_word = scale(key),num_ratings_for_word=n()) %>% 
   tail(20)



subj_code,word,key,rt,scaled_resp_by_subj,num_ratings_by_subj,scaled_resp_by_word,num_ratings_for_word
<int>,<fct>,<int>,<int>,"<dbl[,1]>",<int>,"<dbl[,1]>",<int>
7011,payroll,6,2015,1.4142136,14,0.5773503,3
7011,scaffold,4,1924,0.0000000,14,-0.7071068,2
7011,make,5,1852,0.7071068,14,1.0246951,4
7011,hairpin,5,1986,0.7071068,14,NaN,1
7011,swept,5,1604,0.7071068,14,-1.2247449,4
7011,learning,3,1958,-0.7071068,14,-0.6245384,7
7012,detergent,1,1235,-0.4601950,14,-0.7071068,2
7012,participant,1,960,-0.4601950,14,-0.7071068,2
7012,affectionate,1,1393,-0.4601950,14,NaN,1


## Applying something to multiple columns 

It is often useful to apply something to multiple columns at once. We can do this the long way by using mutate and listing each column and what function we want to apply to it. In cases where we're applying the same function to multiple columns, there's a faster solution using [across()](https://dplyr.tidyverse.org/articles/colwise.html)

Let's get the means of the first four columns for each species listed in the `iris` dataset:

In [42]:
iris %>% head

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa


In [24]:
iris %>%
  group_by(Species) %>% 
  summarise(across(1:4, mean))

Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


The function `mean` can be replaced by any pre-defined function or custom function that you've defined.

In addition to selecting by column index, we can select columns by type or name. Let's see this in action:

In [32]:
iris %>%
  group_by(Species) %>% 
  summarise(across(where(is.numeric), mean)) # select all the numeric columns

Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [48]:
iris %>%
  group_by(Species) %>% 
  summarise(across(contains("Length"), mean)) # select all the columns whose name contains Length

Species,Sepal.Length,Petal.Length
<fct>,<dbl>,<dbl>
setosa,5.006,1.462
versicolor,5.936,4.260
virginica,6.588,5.552


## Running a linear model on each grouping factor

Let's run separate regression models (lm) on each species preicting Sepal.Length from Petal.Length

In [35]:
iris %>% select_if(is.numeric) %>% cor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Sepal.Length,1.0000000,-0.1175698,0.8717538,0.8179411
Sepal.Width,-0.1175698,1.0000000,-0.4284401,-0.3661259
Petal.Length,0.8717538,-0.4284401,1.0000000,0.9628654
Petal.Width,0.8179411,-0.3661259,0.9628654,1.0000000


In [45]:
library(broom)
iris %>% group_by(Species) %>% 
    do(tidy( 
      lm(Sepal.Length ~ Petal.Length, data = .)))

Species,term,estimate,std.error,statistic,p.value
<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
setosa,(Intercept),4.2131682,0.41558877,10.137830,1.614927e-13
setosa,Petal.Length,0.5422926,0.28231526,1.920876,6.069778e-02
versicolor,(Intercept),2.4075231,0.44625834,5.394909,2.075294e-06
versicolor,Petal.Length,0.8282810,0.10413643,7.953806,2.586190e-10
virginica,(Intercept),1.0596591,0.46676645,2.270213,2.772289e-02
virginica,Petal.Length,0.9957386,0.08366764,11.901120,6.297786e-16


Let's try this with something more meaningful. 

In [46]:
library(gapminder)
gapminder %>% head

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


In [60]:
gapminder %>% group_by (year) %>% 
    summarize(lifeExp=mean(lifeExp)) %>% 
    filter(year==min(year) | year==max(year))

year,lifeExp
<int>,<dbl>
1952,49.05762
2007,67.00742


We can see that on average there's been a 18 (!!) year increase in life expectancy between 1952 and 2007. Remarkable progress.
Which countries showed the largest vs. smallest trends in life expectancy in this time? To answer, let's run a separate linear model for each country, predicting life expectancy from year and sort by the coefficient. A large coefficient means a large increase. A small coefficient means a small increase (a negative coefficient means a decrease in life expectancy).

In [53]:
gapminder %>% group_by(country) %>% 
    do(tidy( 
      lm(lifeExp ~ year, data = .))) %>% 
      filter(term=="year") %>% 
      select(country,estimate) %>%
      arrange(-estimate)

country,estimate
<fct>,<dbl>
Oman,0.7721790
Vietnam,0.6716154
Saudi Arabia,0.6496231
Indonesia,0.6346413
Libya,0.6255357
"Yemen, Rep.",0.6054594
West Bank and Gaza,0.6011007
Tunisia,0.5878434
Gambia,0.5818259


```{note}
Whenever we compare slopes (and that's what this coefficient is), we need to be mindful that a shallow slope *could* just mean that life expectancy was already very high at the start. Conversely, a steep slope could come from a having an especially low life expectancy at the start of the timeseries, *or* an especially high life expectancy at the end.
```

The largest increases are in places that had massive industrialization. Smallest increases are places with civil unrest and poor medical care.
Just as a sanity check, let's look at the differences between the minimum and maximum year for the two extremes: Oman and Zimbambe


In [61]:
gapminder %>% group_by(country,year) %>% 
    summarize(lifeExp=mean(lifeExp)) %>% 
    filter(country=="Oman" | country=="Zimbabwe") %>% 
    filter(year==min(year) | year==max(year))

`summarise()` has grouped output by 'country'. You can override using the
`.groups` argument.


country,year,lifeExp
<fct>,<int>,<dbl>
Oman,1952,37.578
Oman,2007,75.640
Zimbabwe,1952,48.451
Zimbabwe,2007,43.487
